In [ ]:
import base64
import requests
import json
import pandas as pd
import pprint
from bs4 import BeautifulSoup as BS

In [ ]:
username = open('../data/rav_user.txt').read()
password = open('../data/rav_pass.txt').read()

How to retrieve the details I need from pattern listings:

- Use code from test notebook to retrieve 10,000 patterns sorted by popularity
- Using id column, write for loop (I think, maybe something else is better) to download patter details (an example using R is on RavelRy github)

retrieve pattern listings:

In [ ]:
# top 1000 patterns sorted by most projects

def get_user_info(query, username, password):
    print(query)
    print(username)
    print(password)
    print('https://api.ravelry.com/patterns/search.json')
    res = requests.get('https://api.ravelry.com/patterns/search.json?sort=projects&page_size=1000&page=1'.format(query),
                       auth=requests.auth.HTTPBasicAuth(username, password))
    return res

In [ ]:
res = get_user_info('', username, password)

In [ ]:
res

In [ ]:
result = json.loads(res.content)

In [ ]:
result['patterns']

In [ ]:
patterns_df = pd.DataFrame(result['patterns'])

In [ ]:
patterns_df.tail()

In [ ]:
result['paginator']

In [ ]:
result['patterns'][0]

In [ ]:
id_list = patterns_df.id.tolist()

In [ ]:
# manual testing to see where the number of projects declines - very long tail!

def get_user_info(query, username, password):
    print(query)
    print(username)
    print(password)
    print('https://api.ravelry.com/patterns/search.json')
    res = requests.get('https://api.ravelry.com/patterns/search.json?query={}&sort=projects&page_size=1000&page=80'.format(query),
                       auth=requests.auth.HTTPBasicAuth(username, password))
    return res

In [ ]:
res = get_user_info('', username, password)

In [ ]:
result = json.loads(res.content)
result['patterns']
patterns2_df = pd.DataFrame(result['patterns'])
patterns2_df.tail()

thinking through code to retrieve results

- pattern ids in list
- create empty df
- define function to pass in username and password
- url to retrieve a single result: base url + patternid as str + .json
- for loop to iterate through id_list
- append each result to df

Initially I was thinking to get top 10,000 but that might not make sense considering the long tail (see notes). Should I cut off at 100 projects? Justification would be that 100 projects is enough to ensure that the pattern appeals to a good number of people. My purpose for this part of the analysis is to determine what type of patterns people like and what common characteristics there are - yarn weights, category, possibly amount of yarn required.

If I take the top 30,000 patterns, I'll need to have a for loop and defined function to iterate over the pages - can I pull the ids straight out into a list from the json in the loop?

Then use list of ids in a for loop to iterate through pattern details and pull results into a dataframe (as in R example in RavelRy project).

notes:

10,000th pattern has 241 projects, while 1st pattern has 23.6k.

12,000th pattern has 206 projects. 15,000th project has 169. I'm actually a little surprised by the length of the tail.

20,000th pattern has 130 projects. 30,000th has 88. 40,000th has 67. 60,000th has 43. 80,000th has 31.

In [ ]:
def get_user_info(username, password):
    pattid_list = []
    for page in range(1, 31):
        print(page)
        res = requests.get('https://api.ravelry.com/patterns/search.json?sort=projects&page_size=1000&page=' + str(page),
                       auth=requests.auth.HTTPBasicAuth(username, password))
        result = json.loads(res.content)
        patterns_df = pd.DataFrame(result['patterns'])
        print(patterns_df.shape)
        pattid_list.append(patterns_df.id.tolist())
    return pattid_list


In [ ]:
pattid_list = get_user_info(username, password)

In [ ]:
print(pattid_list)

In [ ]:
# to flatten list

pattid_flat_list = [str(patt_id) for sublist in pattid_list for patt_id in sublist]

In [ ]:
len(pattid_flat_list)

In [ ]:
# method performed on a string, links each item in the iterable

'+'.join(pattid_flat_list)

In [ ]:
pattid_list[0]

In [ ]:
# turn ids contained in the first list into strings

pattid_list_str = [str(patt_id) for patt_id in pattid_list[0]]

In [ ]:
res = requests.get('https://api.ravelry.com/patterns.json?ids=' + '+'.join(pattid_list_str[:200]), auth=requests.auth.HTTPBasicAuth(username, password))

In [ ]:
res.content

In [ ]:
res = requests.get('https://api.ravelry.com/patterns.json?ids=' + '+'.join(pattid_flat_list), auth=requests.auth.HTTPBasicAuth(username, password))

In [ ]:
res.content

In [ ]:
# not as expected, I'm not sure why - come back to the problem when my brain is fresher

def get_details(username, password, pattids_list):
    patt_details_list = []
    for ids in pattids_list:
        print(ids)
        res = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                           '+'.join(pattid_list_str[:2]), auth=requests.auth.HTTPBasicAuth(username, password))
        result = json.loads(res.content)
        pattdetail_df = pd.DataFrame(result['patterns'])
        print(pattdetail_df.shape)
        patt_details_list.append(pattdetail_df)
    return pd.concat(patt_details_list)
        


In [ ]:
pattdetails_df = get_details(username, password, pattid_list_str)

In [ ]:
# start here, don't do for loop inside function, break list of ids into managable chunks and put that into the request.
# also, the data is wide instead of long (each pattern's data is in a column of 50 rows instead of 1 row across)
# maybe if I go back to the function where I'm pulling the ids, make the page size smaller, then I could pass in
# each list?


res = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                   '+'.join(pattid_list_str[:200]), auth=requests.auth.HTTPBasicAuth(username, password))


Plan of attack

First, write chunk of code to return pattern details for a single id. Should be made into a dataframe and pivoted so that information is in a row instead of a column. (melt, wide_to_long, transpose (pd.t))

Second, expand the code to take multiple ids. Not quite sure how to do this, but maybe once I accomplish the first goal I'll have a better idea. (use range function?)

In [ ]:
# first part of plan accomplished! Glad I was at the point of having a df when I talked with M&M - Mary suggested
# transpose from pandas which accomplished the final part very easily. Magic!

first = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                     str(211562), auth=requests.auth.HTTPBasicAuth(username, password))
first_result = json.loads(first.content)
first_detail_df = pd.DataFrame(first_result['patterns']).T


In [ ]:
first_detail_df

In [ ]:
def second_get_details(username, password, test_id_list):
    